In [2]:
from spotify_keys import *
import pandas as pd
import ast, base64, datetime, json, requests, sqlite3, time

# Primeiro contato com o dataset de Músicas / Captura da relação de Artistas

*   Este projeto utilizará o DataSet disponível [neste link do Kaggle](https://www.kaggle.com/datasets/rodolfofigueroa/spotify-12m-songs), onde há mais de 1 milhão de músicas relacionadas
*   Veja abaixo um exemplo dos dados que temos em mãos

In [3]:
df_tracks = pd.read_csv('million_tracks_dataset.csv')

In [6]:
df_tracks.sample(n=5)

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
160013,6YL3vrIIiMK0GOjlVP8PUj,"15 Hungarian Peasant Songs, BB 79: No. 15. Old...",Bartok: Mikrokosmos (Selection) / Hungarian Pe...,6f5SZrxGlwGPCDQWYlmwMA,"['Béla Bartók', 'Balazs Szokolay']","['5zyNXVd952fWOjkdGHCvPd', '6kJSScipaoehxfUqUT...",19,1,False,0.224,...,0.0361,0.9890,0.845000,0.1010,0.193,85.192,82520,4.0,1993,1993-04-20
1121566,0Jjf0OPQT6qwdr35wTU5Pn,Voices From Heaven,Voices From Heaven,58n4PkR9fV7hQdLqxn5tAh,['Fancy'],['0CZzh950BDSsosMF5Lft5K'],1,1,False,0.681,...,0.0356,0.0179,0.000007,0.3460,0.857,143.051,205947,4.0,2019,2019-12-06
707401,4PI4uq7Lko0ih2N6oVBPDh,You Seem Familiar,Lift,0dfuTkBZtoM7Q9YWc4lW8e,['Akia Uwanda'],['3NTFVOc9RlUt8MLKBDBb5G'],8,1,False,0.714,...,0.0421,0.1700,0.118000,0.1180,0.476,143.939,468393,4.0,2017,2017-05-27
761438,3XsXvPYHEWPlj6W8dyqo27,Handle Ya business,The Cause CD/DVD,1I3QS0Qe1JoHMtpC7lLHiL,['Deebo'],['0Qs1n3ygTM2FKrkm51basT'],7,1,False,0.917,...,0.2040,0.0250,0.000025,0.0679,0.665,94.044,240680,4.0,2005,2005-10-20
690929,5nkSKdCxpEOnC1VQvCJZIn,I Can See It In Your Eyes,Suitcase - Failed Experiments and Trashed Airc...,0nm4DlferXorPSQJJVG9V2,['Guided By Voices'],['4oV5EVJ0XFWsJKoOvdRPvl'],9,2,False,0.356,...,0.0370,0.6980,0.439000,0.2460,0.667,136.111,132747,4.0,2000,2000


Antes de tudo, vamos coletar e separar em uma tabela à parte, o *id* de todos Artistas que contém músicas neste dataset, pois parte de nosso modelo preditivo será feito com base nos dados referentes a eles (seus gêneros e artistas relacionados)


---


Em nosso dataset, apesar da coluna *id_artists* aparentar estar salva em formato de Lista, ela é, na verdade, uma String. Por conta disso, vamos trabalha-la da seguinte forma:

1.   Retirar todos os caracteres que caracterizam uma Lista (mas que não a tornam uma), e manter apenas o conteúdo que nos interessa, isto é, o *id* dos respetivos artistas
2.   Listar os *ids* dos artistas de cada registro, separados pelo limitador (,) e, assim, criar uma lista de fato para cada faixa
3.   Utilizar o método *explode*, capaz de replicar o registro de cada faixa, replicando em registros diferentes uma mesma música, de acordo com 1 ou mais artistas que participam dela
4.   Assim, ao final conseguiremos coletar o *id* de todos os artistas, presentes em todas as faixas



In [7]:
for char in ["[", "]", "'", "\""]:
  df_tracks['artist_ids'] = df_tracks['artist_ids'].str.replace(char, "")

df_tracks['artist_ids'] = df_tracks['artist_ids'].str.split(", ")
id_artists = df_tracks.explode('artist_ids')['artist_ids']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


In [8]:
type(id_artists)

pandas.core.series.Series

*   Deleta eventuais *ids* repetidos, e salva o resultado final em um .csv

In [9]:
id_artists = id_artists.drop_duplicates()

In [10]:
id_artists.shape

(140449,)

In [11]:
id_artists.to_csv('id_artists.csv', index=False) 

# Coletando dados de Artistas e seus Relacionados com a API do Spotify

*   Nesta etapa, vamos coletar, da API do Spotify, dados das nossas Faixas / Artistas que serão necessários à criação do modelo de recomendação, mas que não estão no dataset que temos em mãos
*   Pra começar esta etapa, vamos criar uma função em que geramos o chamado *token*, capaz de autenticar a aplicação e permitir acesso à API

In [ ]:
def get_token():

  # Codifica as chaves de acesso à API do Spotify em Base64

  auth = f"{CLIENT_ID}:{CLIENT_SECRET}"
  auth = auth.encode("ascii")
  auth = base64.b64encode(auth)
  auth = auth.decode("ascii")

  # Cria um request para autorização de acesso à API

  headers = { "Authorization" : "Basic " + auth }
  data = { "grant_type" : "client_credentials" }
  response = requests.post("https://accounts.spotify.com/api/token", headers=headers, data=data)

  # Retorna o token de acesso, ou lança um erro caso a resposta do request seja diferente de 'Sucesso' [200]

  if response.status_code == 200:
    rsp = response.json()
    return rsp['access_token']

  else:
      raise Exception(response.reason)

## Dados de *Artists* (artistas)

### Construindo o DataFrame de Artistas

1.   Vamos aqui coletar os dados de mais de 100 mil Artistas da API do Spotify e salvar no arquivo **spotify_artists**! 
2.   Temos de lidar, porém, com algumas restrições: para manter sua disponibilidade, a API possui formas de manter o controle das aplicações que se utilizam dela, e um desses controles é o [rate limit](https://developer.spotify.com/documentation/web-api/guides/rate-limits/), que impede que uma mesma aplicação faça excessivas requisições dentro de uma janela de 30 segundos.
3.   Assim, temos que construir nossa aplicação tendo em mente que não podemos fazer requests deliberadamente ao servidor.
4.   Para diminuir a quantidade de consultas a serem enviados ao Spotify, vamos utilizar de um recurso presente em alguns *endpoints* da API, onde podemos, com um único request, coletar os dados de até 50 ids (nesse caso, de Artistas) de uma só vez
5.   O método abaixo faz este trabalho, e, quando acontece de atingirmos o chamado *rate limit*, a aplicação fica em stand by por um tempo (estipulado pela própria API), e após esse intervalo, continua novamente a realizar seu trabalho

*OBS.: As próximas células construiram o arquivo .csv e a tabela 
'spotify_artists'*

> Função que busca os Artistas na API

In [ ]:
def get_Spotify_artists(artists_list, token):
  header = { "Authorization" : "Bearer " + token }
  endpoint_url = f"https://api.spotify.com/v1/artists?ids={artists_list}"

  response = requests.get(endpoint_url, headers=header)

  # Retorna com os dados do artista se o request obtiver sucesso
  if response.status_code == 200:
    for endpoint, result in response.json().items() : return result
    

  # Caso atingir a frequência limite de requests, aguarda-se o tempo necessário
  # Logo após, executa-se uma chamada recursiva da função, p/ continuar a coleta dos dados
  elif response.status_code == 429:
    wait = int(response.headers['Retry-After'])
    print(f'Busca pausada por {wait} segundo(s) ...')
    time.sleep(wait)
    token = get_token()
    print(f'Pesquisando ...')
    return get_Spotify_artists(artists_list, token)


  # Retorna com erro para os demais casos
  else : raise Exception(response.reason)

> Chamada da função criada acima, pra construção do DataFrame *spotify_artists*

In [ ]:
token = get_token()
df_concat = list()
i = 0


print(f'Pesquisa de Artistas iniciada')
start = time.time()
    
for i in range(0, len(id_artists), 50):
  
  # Monta um conjunto de 50 ids de Artistas, para serem coletados no Spotify
  pack = id_artists.iloc[i:i+50]['artist_ids'].to_string(header=False, index=False).split('\n')
  ids = ','.join([str(id) for id in pack]) 

  # Executa a função que busca os dados na API e monta em um DataFrame
  df_concat.extend(get_Spotify_artists(ids, token))
  

end = time.time()
elapsed_time = str(datetime.timedelta( seconds = int(end-start) ))
print(f'\nBusca finalizada! Tempo decorrido: {elapsed_time}')

spotify_artists = pd.DataFrame.from_records(df_concat)

Pesquisa de Artistas iniciada

Busca finalizada! Tempo decorrido: 0:04:29




> Veja abaixo uma amostra dos dados de Artistas que temos agora



In [ ]:
spotify_artists

,external_urls,followers,genres,href,id,images,name,popularity,type,uri
0,{'spotify': 'https://open.spotify.com/artist/2...,"{'href': None, 'total': 4902240}","[alternative metal, alternative rock, consciou...",https://api.spotify.com/v1/artists/2d0hyoQ5ynD...,2d0hyoQ5ynDBnkvAbJKORj,"[{'height': 673, 'url': 'https://i.scdn.co/ima...",Rage Against The Machine,69,artist,spotify:artist:2d0hyoQ5ynDBnkvAbJKORj
1,{'spotify': 'https://open.spotify.com/artist/7...,"{'href': None, 'total': 1818298}","[album rock, classic rock, mellow gold, rock, ...",https://api.spotify.com/v1/artists/77tT1kLj6mC...,77tT1kLj6mCWtFNqiOmP9H,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Daryl Hall & John Oates,69,artist,spotify:artist:77tT1kLj6mCWtFNqiOmP9H
2,{'spotify': 'https://open.spotify.com/artist/2...,"{'href': None, 'total': 326189}","[dance pop, europop, new wave pop]",https://api.spotify.com/v1/artists/2U6gqwyl9F3...,2U6gqwyl9F33YxawnFrZG7,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Will Young,50,artist,spotify:artist:2U6gqwyl9F33YxawnFrZG7
3,{'spotify': 'https://open.spotify.com/artist/4...,"{'href': None, 'total': 48172}","[canadian singer-songwriter, classic canadian ...",https://api.spotify.com/v1/artists/4sh4MHP7lhr...,4sh4MHP7lhrSUakxwZzwqz,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Bruce Cockburn,43,artist,spotify:artist:4sh4MHP7lhrSUakxwZzwqz
4,{'spotify': 'https://open.spotify.com/artist/1...,"{'href': None, 'total': 643572}","[art pop, dark pop, ectofolk, lilith, melancho...",https://api.spotify.com/v1/artists/1KsASRNugxU...,1KsASRNugxU85T0u6zSg32,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Tori Amos,54,artist,spotify:artist:1KsASRNugxU85T0u6zSg32
...,...,...,...,...,...,...,...,...,...,...
140444,{'spotify': 'https://open.spotify.com/artist/4...,"{'href': None, 'total': 12}",[],https://api.spotify.com/v1/artists/4CSqFTFO3P0...,4CSqFTFO3P0UEBSJ5Iti5H,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Barry Knight,3,artist,spotify:artist:4CSqFTFO3P0UEBSJ5Iti5H
140445,{'spotify': 'https://open.spotify.com/artist/5...,"{'href': None, 'total': 11220}","[musica jibara, puerto rican folk]",https://api.spotify.com/v1/artists/5TphiK6LsT4...,5TphiK6LsT4X5NOZxq3NJB,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Trio Vegabajeño,38,artist,spotify:artist:5TphiK6LsT4X5NOZxq3NJB
140446,{'spotify': 'https://open.spotify.com/artist/1...,"{'href': None, 'total': 2876}",[],https://api.spotify.com/v1/artists/1oj9F8x44ah...,1oj9F8x44ah5gxYecEj8ch,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",NFL,21,artist,spotify:artist:1oj9F8x44ah5gxYecEj8ch
140447,{'spotify': 'https://open.spotify.com/artist/0...,"{'href': None, 'total': 151}",[austindie],https://api.spotify.com/v1/artists/0UnmV3m91hx...,0UnmV3m91hxEzspbZtoz5b,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Maneja Beto,0,artist,spotify:artist:0UnmV3m91hxEzspbZtoz5b


*   Com a busca de Artistas realizada, foi coletado, para cada um dos ids que tínhamos em mãos, todos os seus dados disponíveis no Spotify, e eles estão mostrados acima
*   Em meio a todos esses dados, porém, existem aqueles que não serão necessários na construção do modelo de predição (como url, imagens, links)
*   Nestes casos, vamos retirá-los da nossa base, pois eles não precisam ser salvos 
*   Além disso, em alguns itens da consulta ao Spotify, *numa mesma coluna* vem preenchidos alguns dados importantes (como o número de followers, isto é, seguidores), porém, junto consigo estão vários outros dados que não serão utilizados em nossa aplicação. Assim, faremos uma 'limpeza' nessas colunas também, capturando delas apenas os dados necessários, e descartando o restante

In [ ]:
spotify_artists.columns

Index(['external_urls', 'followers', 'genres', 'href', 'id', 'images', 'name',
       'popularity', 'type', 'uri'],
      dtype='object')

In [ ]:
spotify_artists = spotify_artists[['id', 'name', 'followers', 'popularity', 'genres']]

In [ ]:
# O método abaixo é responsável por capturar, dentro da coluna especificada, apenas os dados que selecionamos como importantes p/ a aplicação

def capture(content, column):
  # Identifica o tipo de conteúdo em uma coluna (se é uma Lista, um Dicionário, ou uma String), pois possuirão tratamentos diferentes em cada caso

  if isinstance(content, list):
    itens = []
    # Caso seja uma lista, retorna um conjunto de todos os itens da informação selecionada
    for i in content : itens.append(i[column])
    return itens


  elif isinstance(content, dict):
    # Se for um Dicionário, retorna o valor da chave selecionada
    return content[column]
    

  elif isinstance(content, str):
    # Se for uma string, converte-a string em Dicionário, e, logo após, retorna o valor da chave selecionada
    return ast.literal_eval(content)[column]

In [ ]:
wanted_informations = { 'followers' : 'total' }

In [ ]:
for attribute, data in wanted_informations.items():
  spotify_artists[attribute] = spotify_artists[attribute].apply(lambda x : capture(x, data))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  




> Veja agora como ficou nosso conjunto de dados de Artistas



In [ ]:
spotify_artists

,id,name,followers,popularity,genres
0,2d0hyoQ5ynDBnkvAbJKORj,Rage Against The Machine,4902240,69,"[alternative metal, alternative rock, consciou..."
1,77tT1kLj6mCWtFNqiOmP9H,Daryl Hall & John Oates,1818298,69,"[album rock, classic rock, mellow gold, rock, ..."
2,2U6gqwyl9F33YxawnFrZG7,Will Young,326189,50,"[dance pop, europop, new wave pop]"
3,4sh4MHP7lhrSUakxwZzwqz,Bruce Cockburn,48172,43,"[canadian singer-songwriter, classic canadian ..."
4,1KsASRNugxU85T0u6zSg32,Tori Amos,643572,54,"[art pop, dark pop, ectofolk, lilith, melancho..."
...,...,...,...,...,...
140444,4CSqFTFO3P0UEBSJ5Iti5H,Barry Knight,12,3,[]
140445,5TphiK6LsT4X5NOZxq3NJB,Trio Vegabajeño,11220,38,"[musica jibara, puerto rican folk]"
140446,1oj9F8x44ah5gxYecEj8ch,NFL,2876,21,[]
140447,0UnmV3m91hxEzspbZtoz5b,Maneja Beto,151,0,[austindie]


*   Agora temos nossos dados de Artistas num formato razoável, isto é, 
conseguimos buscá-los no Spotify e fazer uma primeira limpeza retirando aquilo que, de antemão, já sabemos que não nos será útil e pode nos economizar gastos (de espaço e processamento)
*   Então vamos salvar este resultado prévio que já temos, que será usado na construção do nosso modelo preditivo

In [ ]:
spotify_artists.to_csv('spotify_artists_dataset.csv', index=False)

### Vinculando os Artistas Relacionados

1.   Um dos tipos mais conhecidos de Sistemas de Recomendação são os chamados [Filtragem Colaborativa Baseada em Itens](https://www.analyticsvidhya.com/blog/2021/05/item-based-collaborative-filtering-build-your-own-recommender-system/#:~:text=Conclusion,based%20on%20user%2Fitem%20similarity.)
2.   Neste tipo de RecSys, um usuário A1 recebe recomendações baseando-se em itens semelhantes àquilo que ele(a) já consumiu/curtiu e comparando com outros usuários, que tenham gosto parecido com aquilo que o A1 já mostrou no passado.
3.   Apesar de a API do Spotify oferecer suporte para buscar músicas curtidas *pelo usuário logado na aplicação (se houver um)*, atualmente, ela não fornece a mesma visão de toda uma comunidade, isto é: é possível implementar uma solução/app que busca as faixas curtidas pelo usuário logado na sessão, porém, não conseguimos comparar, diretamente, suas preferências com os demais usuários da plataforma.
4.   A API, porém, nos fornece uma informação próxima a essa: boa parte dos artistas de seu catálogo possuem o registro dos chamados '[Artistas Relacionados](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-an-artists-related-artists)' (Related Artists), que indica, justamente, para cada Artista, quais outros cantores seus fãs também costumam ouvir .
5.   Usaremos, então, desta informação disponível para o desenvolvimento do nosso modelo de predição, e o método abaixo faz essa busca.

---

6.   Diferente, porém, do que aconteceu anteriormente, para a busca de Artistas Relacionados, não conseguimos fazer um request único que coleta um conjunto de 50 ids, mas sim, esta consulta deverá ser feita artista por artista 
7.   Implantaremos a mesma sistemática anterior que contorna o problema de rate limit, porém, agora, *executaremos a coleta de forma gradativa*, para que nosso request não atinja o limite de "Gateway Timeout" (pois seriam mais de 100.000 requests feitos ao mesmo tempo, caso tentassemos buscar tudo de uma única vez)
8.   Para isso, pegaremos grupos de 5.000 artistas (controlados pelas variáveis *start_point* e *artists_pack* abaixo), e iremos popular nosso conjunto de dados de Artistas Relacionados aos poucos.

*OBS.: As próximas células construiram o arquivo .csv e a tabela 
'related_artists'*

In [ ]:
# Vincula o dataset de Artistas (salvo anteriormente) na variável 'spotify_artists' se ela não estiver carregada na sessão do notebook
spotify_artists = pd.read_csv('spotify_artists_dataset.csv')

> Função que busca os Artistas Relacionados na API

In [ ]:
def get_Spotify_related_artists(artist_id, token):
  header = { "Authorization" : "Bearer " + token }
  endpoint_url = f"https://api.spotify.com/v1/artists/{artist_id}/related-artists"

  response = requests.get(endpoint_url, headers=header)
  
  # Retorna com os dados do artista se o request obtiver sucesso
  if response.status_code == 200:
    for endpoint, result in response.json().items():
      result.insert(0, artist_id)
      return result

    
  # Caso atingir a frequência limite de requests, aguarda-se o tempo necessário
  # Logo após, executa-se uma chamada recursiva da função, p/ continuar a coleta dos dados
  if response.status_code == 429:
    wait = int(response.headers['Retry-After'])
    print(f'Busca pausada por {wait} segundo(s) ...')
    time.sleep(wait)
    token = get_token()
    print(f'Pesquisando ...')
    return get_Spotify_related_artists(artist_id, token)


  # Retorna com erro nos demais casos
  else : raise Exception(response.reason)

In [ ]:
new_related_artists = pd.DataFrame()
related_artists_dataset = pd.read_csv('related_artists_dataset.csv', converters={ 'related_ids': eval , 'related_names': eval , 'related_genres': eval })

> Chamada da função criada acima, pra *construção gradativa* do DataFrame 'related_artists'

**ATENÇÃO:  Controle a execução de consulta aos Artistas Relacionados através da variável '*start_point*' abaixo, colocando o nº do index a partir do qual deve prosseguir a busca**

In [ ]:
start_point = 140000
artists_pack = id_artists.iloc[start_point : start_point + 5000]

In [ ]:
token = get_token()
pack_data = list()

# Aqui nosso DataFrame de Artistas Relacionados será populado com um conjunto de 5.000 artistas buscados da API (essa quantidade depende da variável 'artists_pack' acima)
print(f'Pesquisa de Artistas Relacionados iniciada')
start = time.time()

pack_data.extend(artists_pack['artist_ids'].apply(lambda x : get_Spotify_related_artists(x, token)))

end = time.time()
elapsed_time = str(datetime.timedelta( seconds = int(end-start) ))
print(f'\nBusca finalizada! Tempo decorrido: {elapsed_time}')

new_related_artists = new_related_artists.append(pd.DataFrame.from_records(pack_data), ignore_index = True)

Pesquisa de Artistas Relacionados iniciada
Busca pausada por 1 segundo(s) ...
Pesquisando ...
Busca pausada por 1 segundo(s) ...
Pesquisando ...
Busca pausada por 1 segundo(s) ...
Pesquisando ...
Busca pausada por 1 segundo(s) ...
Pesquisando ...

Busca finalizada! Tempo decorrido: 0:00:36


> Observe abaixo como vieram os dados que buscamos da API

In [ ]:
new_related_artists

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,6rB1jv0emRIkbLluXskjt9,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,5tg5JDw7tQiZdJCShs9rk9,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...
2,5kNXUEveFE9nWmio1YoFek,{'external_urls': {'spotify': 'https://open.sp...,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,2i6trCPrjCF625qTXCK0L2,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,1zcTB8gtjbKxJmluk0amve,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,4CSqFTFO3P0UEBSJ5Iti5H,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
445,5TphiK6LsT4X5NOZxq3NJB,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...
446,1oj9F8x44ah5gxYecEj8ch,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
447,0UnmV3m91hxEzspbZtoz5b,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'spotify': 'https://open.sp...,{'external_urls': {'sp

Precisaremos ajustar este DataFrame de Artistas Relacionados para nossas necessidades antes de salvá-lo em um .csv, e, pra isso, vamos fazê-lo conforme abaixo



---


*   Ajustaremos o nome da coluna de artista 'principal'
*   Agregaremos todos os artistas relacionados numa única coluna
*   Selecionaremos, ao final, somente as colunas necessárias, descartando o restante

In [ ]:
# Este método une todos os artistas relacionados de um determinado id numa lista única, que será salva em uma coluna à parte

def aggregate_related(artist):
  related = list()
  for i in range(1, 21):
     if pd.notna(artist[i]) : related.append(artist[i])

  return related

In [ ]:
new_related_artists.rename(columns = {0 : 'id'}, inplace = True)
new_related_artists['related'] = new_related_artists.apply(aggregate_related, axis = 1)
new_related_artists = new_related_artists[['id', 'related']]

In [ ]:
new_related_artists

,id,related
0,6rB1jv0emRIkbLluXskjt9,[]
1,5tg5JDw7tQiZdJCShs9rk9,[{'external_urls': {'spotify': 'https://open.s...
2,5kNXUEveFE9nWmio1YoFek,[{'external_urls': {'spotify': 'https://open.s...
3,2i6trCPrjCF625qTXCK0L2,[]
4,1zcTB8gtjbKxJmluk0amve,[{'external_urls': {'spotify': 'https://open.s...
...,...,...
444,4CSqFTFO3P0UEBSJ5Iti5H,[]
445,5TphiK6LsT4X5NOZxq3NJB,[{'external_urls': {'spotify': 'https://open.s...
446,1oj9F8x44ah5gxYecEj8ch,[]
447,0UnmV3m91hxEzspbZtoz5b,[{'external_urls': {'spotify': 'https://open.s...


*   Vamos agora replicar a coluna dos Artistas Relacionados, para que possamos dividir, em colunas distintas, os seus dados de id, nome e gênero, e assim facilitar sua manipulação

In [ ]:
for attribute in ['related_ids', 'related_names', 'related_genres'] : new_related_artists[attribute] = new_related_artists['related']

*   Da mesma forma que foi feito no DataFrame de Artistas, vamos limpar os dados advindos da consulta, retirando colunas e extraindo algumas informações que são desnecessários ao modelo, e que não precisam ser salvos e consumir espaço em memória


In [ ]:
new_related_artists.columns

Index(['id', 'related', 'related_ids', 'related_names', 'related_genres'], dtype='object')

In [ ]:
new_related_artists = new_related_artists[['id', 'related_ids', 'related_names', 'related_genres']]

In [ ]:
wanted_informations = { 'related_ids' : 'id', 'related_names' : 'name', 'related_genres' : 'genres' }

In [ ]:
for attribute, data in wanted_informations.items():
  new_related_artists[attribute] = new_related_artists[attribute].apply(lambda x : capture(x, data))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


*   Agora vamos juntar todos os gêneros, de todos os artistas relacionados, numa lista única
*   Além disso, com a integração de diversos artistas relacionados, alguns gêneros irão naturalmente se repetir. Como essa repetição é desnecessária, vamos excluir os gêneros relacionados que estejam repetidos para um mesmo artista

In [ ]:
def clean_related(genres):
  related = [item for conjunct in genres for item in conjunct]
  related = list(dict.fromkeys(related))

  return related

In [ ]:
new_related_artists['related_genres'] = new_related_artists['related_genres'].apply(clean_related)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
new_related_artists

,id,related_ids,related_names,related_genres
0,6rB1jv0emRIkbLluXskjt9,[],[],[]
1,5tg5JDw7tQiZdJCShs9rk9,"[4y5IK8VBswkl40ltpHI5o9, 6wrQP6EAQRd8GLl9tilJd...","[Almighty Suspect, Conradfrmdaaves, Peso Da Go...","[cali rap, west coast trap, pop rap, detroit t..."
2,5kNXUEveFE9nWmio1YoFek,[0bDFIu2L4jBTpJ1Z6v6Bsu],[Tyburn Saints],[]
3,2i6trCPrjCF625qTXCK0L2,[],[],[]
4,1zcTB8gtjbKxJmluk0amve,"[5HQ8niC4OrnYWqZFvgaiR2, 7Dqkr5jq8RjsIUP5hlnwc...","[Mattic, Lex (de Kalhex), Pete Flux, Alcynoos,...","[jazz boom bap, chillhop, lo-fi jazzhop, focus..."
...,...,...,...,...
444,4CSqFTFO3P0UEBSJ5Iti5H,[],[],[]
445,5TphiK6LsT4X5NOZxq3NJB,"[1ekMcZuh9gTNEUw6q6M8WO, 2YWu923Xc47UsMvUKK5Fg...","[Cuarteto Mayari, Felipe Rodriguez, Las Tres G...","[puerto rican folk, musica jibara, bolero, cla..."
446,1oj9F8x44ah5gxYecEj8ch,[],[],[]
447,0UnmV3m91hxEzspbZtoz5b,"[2rQpzaWZ5jaOEBfXfYqKZ9, 35dJmO7Fn6SDDbO39YA8S...","[Seaflea, Fivehead, Pretty Please, The Addicti...","[austindie, rochester ny indie, atlanta punk, ..."


*   Por fim, com esta relação 'limpa', vamos agregá-la no dataset de Artistas Relacionados, salvando-a e integrando-a sempre que finalizar uma etapa da coleta

In [ ]:
related_artists = related_artists_dataset.append(new_related_artists)

> Veja agora como ficou nosso conjunto de dados de Artistas Relacionados

In [ ]:
related_artists

,id,related_ids,related_names,related_genres
0,2d0hyoQ5ynDBnkvAbJKORj,"[2ziB7fzrXBoh1HUPS6sVFn, 6GbCJZrI318Ybm8mY36Of...","[Audioslave, Faith No More, Prophets Of Rage, ...","[alternative metal, alternative rock, nu metal..."
1,77tT1kLj6mCWtFNqiOmP9H,"[3Y3xIwWyq5wnNHPp5gPjOW, 7A9yZMTrFZcgEWAX2kBfK...","[Kenny Loggins, Huey Lewis & The News, Christo...","[album rock, classic rock, mellow gold, new wa..."
2,2U6gqwyl9F33YxawnFrZG7,"[1XgFuvRd7r5g0h844A5ZUQ, 17UkABEasVRlCcIFZ3wHb...","[Take That, Steps, Gary Barlow, Daniel Bedingf...","[boy band, dance pop, europop, bubblegum dance..."
3,4sh4MHP7lhrSUakxwZzwqz,"[4PM86aECDhcdwuJNZNrR22, 5R6GD31ZP8YPGIlt73Mad...","[Chris Smither, Robbie Robertson, Greg Brown, ...","[acoustic blues, alternative country, country ..."
4,1KsASRNugxU85T0u6zSg32,"[3X0tJzVYoWlfjLYI0Ridsw, 76oeXwztPqAxVg9oqozK3...","[Suzanne Vega, Heather Nova, Aimee Mann, Natal...","[lilith, new wave pop, permanent wave, pop roc..."
...,...,...,...,...
140444,4CSqFTFO3P0UEBSJ5Iti5H,[],[],[]
140445,5TphiK6LsT4X5NOZxq3NJB,"[1ekMcZuh9gTNEUw6q6M8WO, 2YWu923Xc47UsMvUKK5Fg...","[Cuarteto Mayari, Felipe Rodriguez, Las Tres G...","[puerto rican folk, musica jibara, bolero, cla..."
140446,1oj9F8x44ah5gxYecEj8ch,[],[],[]
140447,0UnmV3m91hxEzspbZtoz5b,"[2rQpzaWZ5jaOEBfXfYqKZ9, 35dJmO7Fn6SDDbO39YA8S...","[Seaflea, Fivehead, Pretty Please, The Addicti...","[austindie, rochester ny indie, atlanta punk, ..."


In [ ]:
related_artists.duplicated(subset=['id']).sum()

0

In [ ]:
related_artists.to_csv('new_related_artists_dataset.csv', index=False)

#### **IMPORTANTE**: o arquivo salvo aqui como '*new related artists dataset*' será reutilizado neste mesmo notebook como '*related artists dataset*' posteriormente (já que ele tem salvo todos os artistas relacionados pesquisados até o momento), a fim de continuar a coleta, até que ela seja finalizada

## Dados de *Tracks* (faixas)

*   Veja abaixo uma pequena amostra das Faixas que temos em nosso dataset
*   Perçeba que existem vários dados, chamados de **audio features**, que representam as músicas... vamos usar estes dados p/ calcular a semelhança entre elas (danceability, acousticness, energy, ...)

In [ ]:
df_tracks.sample(n=5)

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
128816,1SkuilXmIIHTSHXCUHzGUf,Freedom,The Sails of Self,0aoaTPqX1fdh7YaJQgcvyD,['Rising Appalachia'],['3I6e2ZqqoxQhXc9z7Tp5ci'],2,1,False,0.595,...,0.0637,0.9330,0.000,0.3170,0.446,93.857,139760,4.0,2010,2010-04-23
511393,7K2kssj1VQjrshWPdNrqQt,Intaglio Waltzes,"John PhilipSousa.: Music For Wind Band, Vol. 7...",7sbRE2beqTVr2cEdF1lQxw,"['Royal Artillery Band', 'John Philip Sousa']","['3IzZdcF86S2YxqKkdbVvQG', '6jNyNAMv2gNLnfaP0b...",6,1,False,0.149,...,0.0359,0.9630,0.430,0.0563,0.185,75.634,524107,3.0,2008,2008-12-16
336053,5EPcaxU4jQsjPAQ1AS0yG1,Intergalactic Talk,"The Next Mission, Pt. 2",3gMRi9b9HeWryG5m9ghFKt,['Tor.Ma In DuB'],['6hJVyOMMo1A3XN8yz8neiy'],2,1,False,0.502,...,0.0401,0.0051,0.847,0.2230,0.410,157.978,419491,3.0,2012,2012-08-20
827330,6W3DPXpKDEVSRJWYqLsQov,In The Garden,After The Sunrise,22lywHrCVm37xy5MsTOOjX,['Bill Harrell'],['5pJsZK3VaSayBIVHuyAQHQ'],10,1,False,0.451,...,0.0409,0.9290,0.000,0.0928,0.280,125.620,150760,4.0,2011,2011-10-30
1114764,34oRLdKVxdLKu0wniTmHmI,Gonna Find Me,Flies Like The Wind,0o0G0dxXnlwbWv5wtFqH5l,['Dom Bianco'],['4YPs5J2NVL5zhyfTEjEGpN'],6,1,False,0.613,...,0.0265,0.2660,0.907,0.0668,0.646,100.094,198973,4.0,2005,2005-06-14


*   Além destes dados, que nós já temos em nosso dataset, o Spotify disponibiliza também, a chamada **audio analysis** que, assim como os *audio features*, 'traduzem' a estrutura das músicas em números
*   Da mesma forma que acontece com os artistas relacionados, a API disponibiliza a consulta de *audio analysis* de cada faixa individualmente
*   Veja abaixo o exemplo de *audio analysis* de uma única faixa






In [ ]:
header = { "Authorization" : "Bearer " + get_token() }

id = '6JZLVLOYVHEOwqZK0ezTdf'
url = f"https://api.spotify.com/v1/audio-analysis/{id}"

response = requests.get(url, headers=header)

# Retorna com os dados do artista se o request obtiver sucesso
if response.status_code == 200:
  for endpoint, result in response.json().items() : print (result)

{'analyzer_version': '4.0.0', 'platform': 'Linux', 'detailed_status': 'OK', 'status_code': 0, 'timestamp': 1584953651, 'analysis_time': 11.09301, 'input_process': 'libvorbisfile L+R 44100->22050'}
{'num_samples': 7299300, 'duration': 331.034, 'sample_md5': '', 'offset_seconds': 0, 'window_seconds': 0, 'analysis_sample_rate': 22050, 'analysis_channels': 1, 'end_of_fade_in': 0.0, 'start_of_fade_out': 318.87964, 'loudness': -10.641, 'tempo': 87.012, 'tempo_confidence': 0.667, 'time_signature': 4, 'time_signature_confidence': 0.658, 'key': 7, 'key_confidence': 0.899, 'mode': 0, 'mode_confidence': 0.797, 'codestring': 'eJxVnAmW5aquRKeSQwDRz39iFTuEj13vr3V_meMGhBQKNeRafRb93_krfzHWHGus-tei_O3ovUUb_a_Wrf-suiLOWX91nvnXS-woe4eemoVfW60x118rS2PtRI8Z_a-dNv_WGvpZD__13rcu_c0Rf0Ojf6F_xV767NA3mUUtdbT9N-fef22d3cbY629FC132VfXJ8bfL1KxK36fX0zTfWTbXZ5bSxtK1JjLKGbs3zfB05jVWO3txebQmTaSW0kfTc3xL_50zauN6a6J1xxiz7flXIzS12fR7O-X8Vb1SL5GItPKjO9se80_zbFF70-9d0tB1l0Tnrn91dL1_aS5aFPfP7dnr_-tTDfGuYAJ1lSmhNL7dtajSEIWWW4

*   Como podemos ver, uma única música gera um resultado relativamente grande quando buscamos seus dados de *audio analysis*
*   Isso porque estes dados são, na verdade, a decodificação de cada pedacinho da música, incluindo seu ritmo, timbre e tom ([para saber mais, clique aqui](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-audio-analysis))
*   Considerando que vamos trabalhar com + 1 milhão de faixas, de gêneros muito variados entre si, estariamos lidando com uma quantidade absurda de dados, que acarretaria em um custo alto de processamento e de consultas à API, e talvez teríamos um ganho apenas marginal em nossa aplicação
*   Portanto, para a criação desse nosso modelo preditivo, vamos nos limitar aos dados de *audio features* (que já temos disponibilizados no dataset do Kaggle), apesar da API disponibilizar também estes dados de *audio analysis*






# Salvando todos dados no Banco de Dados SQLite3

1.   **Criando o banco e gerando uma conexão com ele**



In [12]:
conn = sqlite3.connect('recsys.db')

2.   **Salvando as tabelas de Faixas, Artistas e Artistas Relacionados**

Tabela de FAIXAS

In [ ]:
df_tracks.to_sql(name='tracks', con=conn)

Tabela de ARTISTAS

In [ ]:
spotify_artists.to_sql(name='artists', con=conn)

Tabela de ARTISTAS RELACIONADOS

> Para salvar a relação de Artistas Relacionados no nosso Banco de Dados, é necessário transformar em String os dados que estão no formato Lista no .csv

In [ ]:
related_artists_copy = related_artists_dataset.copy()

In [ ]:
for column in ['related_ids', 'related_names', 'related_genres'] : related_artists_copy[column] = related_artists_copy[column].apply(lambda x: ', '.join([str(i) for i in x]))

In [ ]:
related_artists_copy.to_sql(name='related_artists', con=conn)

3.   **Consultas gerais ao banco criado**



In [13]:
df = pd.read_sql('SELECT * FROM tracks', con=conn, index_col='index')

In [14]:
df

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
index,,,,,,,,,,,,,,,,,,,,,
0,7lmeHLHBe4nmXzuXc0HDjk,Testify,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],1,1,0,0.470,...,0.0727,0.02610,0.000011,0.3560,0.503,117.906,210133,4.0,1999,1999-11-02
1,1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],2,1,1,0.599,...,0.1880,0.01290,0.000071,0.1550,0.489,103.680,206200,4.0,1999,1999-11-02
2,1hR0fIFK2qRG3f3RF70pb7,Calm Like a Bomb,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],3,1,0,0.315,...,0.4830,0.02340,0.000002,0.1220,0.370,149.749,298893,4.0,1999,1999-11-02
3,2lbASgTSoDO7MTuLAXlTW0,Mic Check,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],4,1,1,0.440,...,0.2370,0.16300,0.000004,0.1210,0.574,96.752,213640,4.0,1999,1999-11-02
4,1MQTmpYOZ6fcMQc56Hdo7T,Sleep Now In the Fire,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],5,1,0,0.426,...,0.0701,0.00162,0.105000,0.0789,0.539,127.059,205600,4.0,1999,1999-11-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1204020,0EsMifwUmMfJZxzoMPXJKZ,Gospel of Juke,Notch - EP,38O5Ys0W9PFS5K7dMb7yKb,['FVLCRVM'],['7AjItKsRnEYRSiBt2OxK1y'],2,1,0,0.264,...,0.0672,0.00935,0.002240,0.3370,0.415,159.586,276213,4.0,2014,2014-01-09
1204021,2WSc2TB1CSJgGE0PEzVeiu,Prism Visions,Notch - EP,38O5Ys0W9PFS5K7dMb7yKb,['FVLCRVM'],['7AjItKsRnEYRSiBt2OxK1y'],3,1,0,0.796,...,0.0883,0.10400,0.644000,0.0749,0.781,121.980,363179,4.0,2014,2014-01-09
1204022,6iProIgUe3ETpO6UT0v5Hg,Tokyo 360,Notch - EP,38O5Ys0W9PFS5K7dMb7yKb,['FVLCRVM'],['7AjItKsRnEYRSiBt2OxK1y'],4,1,0,0.785,...,0.0564,0.03040,0.918000,0.0664,0.467,121.996,385335,4.0,2014,2014-01-09
